In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pickle

Using TensorFlow backend.


In [2]:
lines = pd.read_pickle('eng-hin-parallel.pickle')

In [3]:
lines = lines[:10000]

In [4]:
lines.shape

(10000, 2)

In [5]:
lines.sample(10)

,Eng,Hin
2639,the four of clubs,<START> चिड़ी का चौवा <END>
2613,click to move,<START> बढ़ने के लिये क्लिक करें <END>
3491,two suits,<START> दो सूट <END>
3175,click to move,<START> बढ़ने के लिये क्लिक करें <END>
4364,base card king,<START> आधार पत्ताः बादशाह <END>
2957,deal new cards from the deck,<START> डेक में से नया पत्ता चलें <END>
5141,move a off the board,<START> को पट्ट से बाहर ले जाएँ <END>
5846,place the eight of spades next to the seven of...,<START> हुकुम की अट्ठा के बगल में हुकुम के सत्...
5933,show,<START> दिखाएँ <END>
7977,options,<START> विकल्प <END>


In [6]:
with open('embeddings.en', 'rb') as f:
    eng_summary = pickle.load(f)
    
with open('embeddings.hi', 'rb') as f:
    hin_summary = pickle.load(f)

In [7]:
all_eng_words = eng_summary['dictionary'].keys()
all_hin_words = hin_summary['dictionary'].keys()

In [8]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Eng])
max_len_src

41

In [9]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Hin])
max_len_tar

40

In [10]:
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hin_words)
(num_encoder_tokens, num_decoder_tokens)

(2889, 3122)

In [11]:
lines = shuffle(lines)
lines.head(10)

,Eng,Hin
3229,ace of diamonds,<START> ईंट का इक्का <END>
6586,cut,<START> काटें काटो काटो <END>
2297,no redeals,<START> कोई डील फिर नहीं <END>
5392,pileon,<START> पाइलेअन <END>
6013,profile plugins,<START> छैतिज पैमाना <END>
4558,remove the kings,<START> बादशाह मिटाएँ <END>
2779,remove the seven of spades,<START> हुकुम का दहला हटाएँ <END>
4017,deals left a,<START> बायीँ डीलः <END>
2965,undo until there are enough cards to fill all ...,<START> सभी ढेर भरने लायक पत्ते होते तक पहले ज...
3477,the four of diamonds,<START> ईंट का चव्वा <END>


In [12]:
X, y = lines.Eng, lines.Hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape, X_test.shape

((9000,), (1000,))

In [13]:
def generate_batch(X=X_train, y=y_train, batch_size=128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
            
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = eng_summary['dictionary'][word]
                    
                for t, word in enumerate(target_text.split()):
                    if t < len(target_text.split())-1:
                        decoder_input_data[i, t] = hin_summary['dictionary'][word]
                        
                    if t > 0:
                        decoder_target_data[i, t-1, hin_summary['dictionary'][word]] = 1
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [14]:
latent_dim = 128

In [15]:
encoder_inputs = Input(shape=(None, ), name='Inputs')
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True, weights=[eng_summary['embeddings']], name='English_Embeddings')(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='Encoder_LSTM')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
decoder_inputs = Input(shape=(None, ), name='Outputs')
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True, weights=[hin_summary['embeddings']], name='Hindi_Embeddings')
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='Decoder_LSTM')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [19]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

In [20]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
140/140 [==============================] - 97s 694ms/step - loss: 5.2009 - acc: 0.2445 - val_loss: 4.5864 - val_acc: 0.2568
Epoch 2/10
140/140 [==============================] - 97s 692ms/step - loss: 4.1936 - acc: 0.2904 - val_loss: 3.9592 - val_acc: 0.3257
Epoch 3/10
140/140 [==============================] - 98s 697ms/step - loss: 3.6750 - acc: 0.3629 - val_loss: 3.5670 - val_acc: 0.3714
Epoch 4/10
140/140 [==============================] - 96s 687ms/step - loss: 3.2976 - acc: 0.4128 - val_loss: 3.2641 - val_acc: 0.4127
Epoch 5/10
140/140 [==============================] - 98s 699ms/step - loss: 2.9723 - acc: 0.4644 - val_loss: 2.9448 - val_acc: 0.4760
Epoch 6/10
140/140 [==============================] - 98s 701ms/step - loss: 2.6647 - acc: 0.5077 - val_loss: 2.6904 - val_acc: 0.5152
Epoch 7/10
140/140 [==============================] - 100s 711ms

In [21]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [31]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hin_summary['dictionary']['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = hin_summary['reverse_dictionary'][sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [32]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [64]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][7:-5])
print('Predicted Hindi Translation:', decoded_sentence[:-5])

Input English sentence: too many selectable children
Actual Hindi Translation:  बहुत अधिक चयनीय शिशु हैं 
Predicted Hindi Translation:  बहुत अधिक चयनीय शिशु हैं 
